<a href="https://colab.research.google.com/github/scarlettjm/MOMA-Acquisitions/blob/main/Final_PFCH_Moldovan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Function Library**

---


### **clean_nationality_column**

**Input**:

* `df` (pd.DataFrame): A DataFrame with a 'Nationality' column.

**Function**:

* Cleans the 'Nationality' column by:

  * Replacing empty values or inconsistencies with 'Nationality Unknown'.
  * Standardizing known spelling errors or variations (e.g., 'English' → 'British').

**Output**:
* `pd.DataFrame`: The updated DataFrame with a cleaned 'Nationality' column.

---
### **assign_decade**
**Input**:

* `date (datetime)`: A datetime object representing the acquisition date.

**Function**:

* Assigns a decade (e.g., '1930s') based on the year of the acquisition date.
* Returns a string representing the decade if the year is within 1930–2024, otherwise returns None.

**Output**:

* `str`: The decade string (e.g., '1930s') or None if the year is not within the range.

---
### **group_by_decade**
**Input**:

* `df` (pd.DataFrame): A DataFrame containing an acquisition date column.

**Function**:

* Groups artworks into decades based on their acquisition dates.
* Uses the assign_decade function to determine the decade for each entry.

**Output**:

* `dict[str, pd.DataFrame]`: A dictionary with decades as keys and corresponding DataFrames as values.

---
### **extract_nationalities**
**Input**:

* `series` (pd.Series): A pandas Series containing strings of nationalities formatted with parentheses (e.g., "(American)").

**Function:**
* Extracts nationalities enclosed in parentheses and filters out unwanted values like "Nationality Unknown".
* Handles lists of multiple nationalities per entry.

**Output**:

* `list[str]`: A list of cleaned nationality strings.

---
### **calculate_percentages**
**Input**:

*
`counts` (Counter): A Counter object with nationalities as keys and their counts as values.

**Function**:

* Calculates percentage representation for each nationality based on its count.
* Total count is used as the denominator to compute percentages.

**Output**:

* `dict[str, float]`: A dictionary with nationalities as keys and their percentage values as floats.

---
### **generate_treemap**
**Input**:

* `data` (pd.DataFrame): A DataFrame with 'Nationality' and 'Percentage' columns.
* `title` (str): Title for the treemap visualization.

**Function**:

* Creates and displays a treemap visualization of nationality percentages.
* Uses matplotlib and squarify for plotting.
* Removes axes for a cleaner look and applies proportional sizing based on percentage values.

**Output**:

* None (displays the treemap).

---
### **top_nationalities**
**Input:**

* `data` (pd.DataFrame): A DataFrame containing nationality percentages for a specific decade.
* `n` (int): Number of top nationalities to select.

**Function**:

* Selects the top `n` nationalities by percentage from the provided DataFrame.
* Sorts values in descending order to ensure top representation.

**Output**:

* `pd.DataFrame`: A DataFrame of the top `n ` nationalities.

---
### **calculate_trend_analysis**
**Input**:

* `data` (pd.DataFrame): A combined DataFrame with percentages for all decades.

**Function**:

* Creates a pivot table to show nationality percentages across decades.
* Calculates the percentage growth for each nationality between the first and last decade.

**Output**:

* `pd.DataFrame`: A DataFrame showing trends and percentage growth for each nationality.

---
### **save_to_excel**
**Input**:

* `filename` (str): Name of the output Excel file.
* `dataframes` (dict): A dictionary where keys are sheet names and values are DataFrames to be written to the workbook.

**Function**:

* Saves multiple DataFrames into an Excel workbook with each DataFrame on a separate sheet.
* Handles saving and formatting using `pandas.ExcelWriter`.

**Output**:

* None (writes the file and downloads it).


## **Project Overview**
> This project analyzes the MoMA Artworks dataset, which contains detailed information about artworks in the Museum of Modern Art’s collection. The dataset includes attributes such as acquisition dates, artist nationalities, and artwork titles, providing a rich resource for exploration.

> The primary focus of this project is on the nationality representation of artists within the collection. The exploration involves cleaning and standardizing the data, grouping artworks by the decade of acquisition, and calculating the percentage representation of different nationalities over time. Visualizations such as treemaps and trend analysis tables are used to uncover patterns and highlight disparities.

> Through this analysis, I aim to identify:

> * The dominant nationalities represented in the collection.
* Trends in acquisition over time, including the growth or decline of representation for specific nationalities.
* Underrepresented regions or countries, offering insights into potential gaps in curatorial practices.

> Ultimately, this project seeks to provide a clearer picture of MoMA's acquisition history, exploring how it reflects broader cultural and institutional trends in the art world.

In [ ]:
'''Install squarify to create tree maps'''
!pip install squarify

In [ ]:
'''Load the necessary libraries'''
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import squarify
from google.colab import files

## **Prior Cleaning**

> Before uploading the Artworks dataset to Colab, I partially cleaned and edited it in Excel. I began by sorting the data by nationality, which revealed thousands of rows with missing nationality information. Over the course of several days, I researched and filled in many of these gaps using resources such as other institutions' records, artist websites, historical archives, and exhibition listings. For collectives based in specific countries, nationality was determined by their country of origin. Similarly, for works created by companies or corporations, nationality was assigned based on their location. Any entries that could not be traced were left blank.

> After addressing the nationality data, I briefly examined the acquisition dates. Acquisition records prior to the 1930s were sparse, with accompanying columns often incomplete and messy. Given the limited utility of this data and the project's scope, all entries from before the 1930s were removed for clarity and consistency.






In [ ]:
'''Load the dataset'''
file_path = "/content/Copy of Artworks 2.csv"
df = pd.read_csv(file_path)
#Verify the first few rows
print(df.head())


                          Title     Artist ConstituentID ArtistBio  \
0                      UNTITLED       Anon         29875       NaN   
1                    Bandit 201  Anonymous         10727       NaN   
2                    Tattoo 103  Anonymous         10727       NaN   
3  A Dessert For Your Breakfast  Anonymous         10727       NaN   
4    Calcium For Your Breakfast  Anonymous         10727       NaN   

  Nationality BeginDate EndDate  Gender              Date  \
0          ()         0       0  (male)  January 25, 2001   
1          ()         0       0      ()              1967   
2          ()         0       0      ()              1967   
3          ()         0       0      ()    (late 1960's?)   
4          ()         0       0      ()               NaN   

                                            Medium  ... OnView  \
0                                     Ink on paper  ...    NaN   
1  Offset lithograph on pressure-sensitive sticker  ...    NaN   
2  Offset lith

<ipython-input-20-05c9f2d36f11>:3: DtypeWarning: Columns (2,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


## **Data Cleaning**

In [ ]:
'''Drop unnecessary columns to focus on relevant data (Acquisition Date and Nationality)'''
columns_to_remove = [
    'ConstituentID', 'ArtistBio', 'Date', 'BeginDate', 'EndDate', 'Gender', 'Medium',
    'Dimensions', 'AccessionNumber', 'Cataloged', 'ObjectID', 'OnView', 'URL', 'ImageURL',
    'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)', 'Length (cm)',
    'Weight (kg)', 'Width (cm)', 'Seat Height (cm)', 'Duration (sec.)', 'CreditLine'
]
df = df.drop(columns=columns_to_remove)

#Verify the remaining columns
print(df.columns)

Index(['Title', 'Artist', 'Nationality', 'Classification', 'Department',
       'DateAcquired'],
      dtype='object')


In [ ]:
'''Clean the Nationality column'''
#Replace inconsistent labels and remove errors
if 'Nationality' in df.columns:
    '''Replace empty parentheses '()' with 'Nationality Unknown' '''
    df['Nationality'] = (
        df['Nationality']
        .str.replace(r'\(\)', '(Nationality Unknown)', regex=True)
        .str.replace(r'(\(Nationality Unknown\))+', '(Nationality Unknown)', regex=True)
        .str.strip()
    )

    #Correct specific errors and standardize spellings
    corrections = {
        r'\(Nationality unknown\)': '(Nationality Unknown)',
        r'\(English\)': '(British)',
        r'\(Ukranian\)': '(Ukrainian)',
        r'\(Lithianian\)': '(Lithuanian)',
    }
    for pattern, replacement in corrections.items():
        df['Nationality'] = df['Nationality'].str.replace(pattern, replacement, regex=True)

#Verify unique values in the Nationality column
print(df['Nationality'].unique())

['(Nationality Unknown)' '(Nationality Unknown) (Nationality Unknown)'
 '(Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown) (Nationality Unknown)'
 ... '(Yugoslav)' '(Zimbabwean)' nan]


In [ ]:
'''Save cleaned dataset to a CSV for reference'''
df.to_csv("cleaned_dataset.csv", index=False)
files.download("cleaned_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''Clean and reformat acquisition dates'''
if 'DateAcquired' in df.columns:
    #Remove rows with missing acquisition dates
    df = df.dropna(subset=['DateAcquired'])

    #Exclude future-dated acquisitions (e.g., 2029)
    df = df[df['DateAcquired'] != '11/19/29']

    #Convert to datetime format for consistent processing
    df['DateAcquired'] = pd.to_datetime(df['DateAcquired'], errors='coerce')

#Verify cleaned acquisition dates
print(df['DateAcquired'].head())

0   2005-05-10
1   2008-10-08
2   2008-10-08
3   2008-10-08
4   2008-10-08
Name: DateAcquired, dtype: datetime64[ns]


## **Refining the data by decade**

In [ ]:
'''Group artworks by decade'''
#Define a function to assign decades based on the year
def assign_decade(date):
    '''Assigns a decade string (e.g., '1930s') based on the year of the date.'''
    if pd.isnull(date):
        return None
    year = date.year
    return f"{year // 10 * 10}s" if 1930 <= year <= 2024 else None

#Apply the decade function to create a new column
df['Decade'] = df['DateAcquired'].apply(assign_decade)

#Group the dataset by decade
artworks_by_decade = {decade: group for decade, group in df.groupby('Decade')}

#Save grouped datasets to CSV for each decade
for decade, group in artworks_by_decade.items():
    group.to_csv(f'artworks_{decade}.csv', index=False)
    files.download(f'artworks_{decade}.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Nationality percentage grouping**

In [ ]:
'''Calculate nationality percentages for each decade (excluding Nationality Unknown)'''
nationality_percentages_by_decade = {}

for decade, group in artworks_by_decade.items():
    #Extract all nationalities while excluding 'Nationality Unknown'
    all_nationalities = group['Nationality'].str.findall(r'\((.*?)\)').apply(
        lambda x: [nat for nat in x if nat != 'Nationality Unknown'] if isinstance(x, list) else []  # Handle non-list values
    ).sum()

    #Count occurrences of each nationality
    nationality_counts = Counter(all_nationalities)
    total = sum(nationality_counts.values())

    if total > 0:
        #Calculate percentage representation for each nationality
        nationality_percentages_by_decade[decade] = pd.DataFrame(
            {
                'Nationality': list(nationality_counts.keys()),
                'Percentage': [(count / total) * 100 for count in nationality_counts.values()]
            }
        ).sort_values(by='Percentage', ascending=False)

        #Save to CSV for visualization or analysis
        nationality_percentages_by_decade[decade].to_csv(f"nationality_percentages_{decade}.csv", index=False)
        files.download(f"nationality_percentages_{decade}.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Visualizing the data**

In [ ]:
'''Create treemaps for each decade based on nationality acquisitions'''
for decade, data in nationality_percentages_by_decade.items():
    plt.figure(figsize=(12, 8))

    #Prepare data for the treemap
    sizes = data['Percentage']
    labels = [f"{row['Nationality']} ({row['Percentage']:.1f}%)" for _, row in data.iterrows()]

    #Create the treemap
    squarify.plot(sizes=sizes, label=labels, alpha=0.8, color=plt.cm.tab20.colors)
    plt.axis('off')
    plt.title(f"{decade} Nationality Acquisition Data Treemap", fontsize=16)

    #Save and display the treemap
    file_name = f"treemap_{decade}.png"
    plt.savefig(file_name, bbox_inches='tight')
    files.download(file_name)
    plt.close()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


## **Creating Detailed Treemaps**
>To create the final treemaps, I used Canva and Illustrator, building upon the initial treemaps generated in Colab. The Colab-generated treemaps served as a foundational layer, providing the proportional representation of nationalities based on acquisition percentages for each decade.

>Using these as a starting point, I enhanced the design and readability by:
* Overlaying updated visual elements: I imported the Colab treemaps into Canva and Illustrator, where I could layer additional design elements such as labels, color gradients, and clearer typography.
* Adding inset maps: Recognizing that some sections of the treemaps were too dense to view clearly, I created inset maps—smaller zoomed-in sections placed alongside the main treemap. These inset maps provided detailed views of areas with smaller nationalities, ensuring nothing was overlooked.
* Customizing decade-specific aesthetics: To reflect the evolving themes and styles of each decade, I incorporated unique color schemes, fonts, and other visual motifs for each treemap. For instance, treemaps for earlier decades have muted, vintage-inspired palettes, while more recent decades feature modern, vibrant colors.

> More detailed visualizations can be found [here](https://drive.google.com/drive/folders/1s51YmYpd_IyVH2bRVd05VIvVwFJQfsre?usp=sharing)



## **Trend Analysis and Dominance**

In [ ]:
'''Perform trend analysis'''
all_decades_data = []
for decade, data in nationality_percentages_by_decade.items():
    #Add a Decade column for easier merging
    data['Decade'] = decade
    all_decades_data.append(data)

#Combine all decades into one DataFrame
combined_data = pd.concat(all_decades_data)

#Create a pivot table with nationalities as rows and decades as columns
trend_table = combined_data.pivot(index='Nationality', columns='Decade', values='Percentage')

#Calculate percentage growth for each nationality across the dataset
trend_table['Growth (%)'] = trend_table.iloc[:, -1] - trend_table.iloc[:, 0]

#Save trend analysis to CSV
trend_table.to_csv("nationality_trends.csv")
files.download("nationality_trends.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Display the trend table for reference
print(trend_table)

Decade           1930s      1940s      1950s      1960s      1970s      1980s  \
Nationality                                                                     
Afghan             NaN        NaN        NaN        NaN        NaN        NaN   
Albanian           NaN        NaN        NaN        NaN        NaN        NaN   
Algerian           NaN        NaN   0.030680        NaN        NaN        NaN   
American     54.222222  48.180899  30.495475  47.941322  60.487914  55.205047   
Argentine          NaN   0.391612   0.536892   0.391486   0.329267   0.074225   
...                ...        ...        ...        ...        ...        ...   
Venezuelan         NaN        NaN   0.015340   0.089997   1.189853   0.027834   
Vietnamese         NaN        NaN        NaN        NaN        NaN        NaN   
Welsh              NaN        NaN        NaN        NaN        NaN        NaN   
Yugoslav           NaN   0.012633        NaN        NaN        NaN        NaN   
Zimbabwean         NaN      

In [ ]:
'''Identify top nationalities for each decade'''
for decade, data in nationality_percentages_by_decade.items():
    #Select the top 5 nationalities based on percentage
    top_5 = data.nlargest(5, 'Percentage')
    top_5.to_csv(f"top_nationalities_{decade}.csv", index=False)
    files.download(f"top_nationalities_{decade}.csv")

    #Print top 5 nationalities for quick review
    print(f"--- Top Nationalities in {decade} ---")
    print(top_5)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--- Top Nationalities in 1930s ---
   Nationality  Percentage Decade
0     American   54.222222  1930s
12      French   16.055556  1930s
15      German    8.166667  1930s
5      British    7.833333  1930s
13     Spanish    3.722222  1930s


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

--- Top Nationalities in 2010s ---
   Nationality  Percentage Decade
0     American   41.066889  2010s
6       German   10.777834  2010s
2       French    5.628586  2010s
13   Argentine    4.982206  2010s
1      British    4.001743  2010s


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

--- Top Nationalities in 2020s ---
                Nationality  Percentage Decade
0                  American   61.833019  2020s
5                   Chinese    5.712492  2020s
20  Trinidad and Tobagonian    4.237288  2020s
13                  British    2.887633  2020s
11                Brazilian    1.914626  2020s


In [ ]:
'''Install necessary libraries for Excel export'''
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 2.6 MB/s eta 0:00:00


## **Comprehensive Workbook of all files**

In [ ]:
'''Save all created CSVs into one Excel workbook'''
import os

output_file = "artworks_combined.xlsx"

#Use ExcelWriter to combine all DataFrames into a single workbook
with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
    #Save the original cleaned dataset
    df.to_excel(writer, sheet_name="Cleaned_Data", index=False)

    #Save grouped datasets by decade
    for decade, group in artworks_by_decade.items():
        group.to_excel(writer, sheet_name=decade, index=False)

    #Save nationality percentages by decade
    for decade, data in nationality_percentages_by_decade.items():
        data.to_excel(writer, sheet_name=f"{decade}_Percentages", index=False)

    #Save top nationalities by decade
    for decade, data in nationality_percentages_by_decade.items():
        #Select the top 5 nationalities for each decade
        top_5 = data.nlargest(5, 'Percentage')
        top_5.to_excel(writer, sheet_name=f"{decade}_Top5", index=False)

    #Save trend analysis
    trend_table.to_excel(writer, sheet_name="Trend_Analysis", index=True)

#Download the combined workbook
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Findings**
---
Of the 132 nationalities present in the cleaned dataset, only a subset, including American, Brazilian, Swedish, Chilean, Cuban, Polish, Dutch, Israeli, Belgian, Norwegian, Austrian, Italian, Swiss, Mexican, Russian, Spanish, British, German, and French, displayed notable trends of growth or decline in acquisition data.

---


**Dominance of American Art**

The treemaps and trend analysis vividly illustrated the sustained dominance of American artists in MoMA’s collection. Starting at 54.22% in the 1930s and increasing to 61.83% in the 2020s, American representation consistently occupied the largest portions of the treemaps. This dominance reflects a strong institutional preference or accessibility to American artworks, possibly driven by geographic proximity, cultural influence, or the prominence of American art movements throughout the 20th and 21st centuries.


---


**Emerging Trends in Latin America**

The data also revealed a steady, albeit smaller, growth in the representation of Cuban, Chilean, and Brazilian artists. These nationalities occupied modest but expanding areas in the visualizations, reflecting a potential growing interest in Latin American art within major institutions. This trend may indicate a shift toward greater inclusion of diverse cultural narratives.

---

**Decline of Traditional European Dominance**

European nationalities such as German, Spanish, Italian, and British saw a decline in representation over the decades. The shrinking proportions of these regions on the treemaps suggested a gradual move away from traditional Western European dominance. While these nationalities remain significant, their reduced presence hints at evolving curatorial priorities.

---

**Underrepresentation in Global Context**

Analysis starkly highlighted regions that remain significantly underrepresented in MoMA's acquisitions:

* Africa: Nationalities from African countries, such as Zimbabwe, appeared sporadically, and large portions of the continent were absent entirely, underscoring historical neglect.
* Oceania: Countries from Oceania were largely missing from the treemaps, indicating minimal institutional focus on this region.
Asia: Southeast Asian and Central Asian nationalities barely featured, reflecting systemic gaps in acquisition efforts for these areas.
* Latin America: While some growth was noted in countries like Brazil and Cuba, others, such as Venezuela, remained marginalized.

---

**Visual Impact of Treemaps**

The treemaps not only provided a proportional understanding of the data but also offered a powerful visual demonstration of dominance and absence:

* Large, consistent blocks representing American art dominated every decade.
* Smaller, emerging blocks for nations like Chile, Cuba, and Brazil visually captured their increasing influence.
* Entirely blank or negligible sections for Africa, Oceania, and parts of Asia highlighted glaring gaps in representation.

---

**Broader Implications**

Despite a minor shift, the data highlights a significant lack of representation for artists from Africa, Oceania, and parts of Asia and Latin America in MoMA's acquisitions. African countries, such as Zimbabwe, appear sporadically, and much of the continent is absent, reflecting historical neglect. Oceania is similarly underrepresented, while Southeast Asian nations and Central Asian regions barely feature. Despite some growth in Latin American representation, countries like Venezuela remain marginalized.

This lack of growth reflects broader challenges, including historical underrepresentation, geopolitical factors, and potential barriers to accessibility in the global art market. The disparity might also stem from systemic biases in curatorial practices that historically favored Western or established art traditions, leaving other regions underrepresented. Overall, these trends highlight both areas of strong representation and gaps where specific regions or nationalities may be underrepresented, offering a potential roadmap for curatorial diversification and inclusion efforts.


---

**Opportunities for Change**

These findings serve as both a visual and data-driven roadmap for curatorial diversification and inclusion efforts. They emphasize the need for institutions like MoMA to expand acquisitions to include a broader spectrum of global art, ensuring equitable representation of all cultural voices. This insight is crucial for fostering a more inclusive and representative narrative of modern and contemporary art.